# MCP Agents Educational Demo

**START HERE!** This is the basic demo. Run `mcp_llm_agent.ipynb` after this one.

This notebook demonstrates how to interact with MCP (Model Context Protocol) servers:
- **Code Server**: File operations and code execution
- **Database Server**: SQLite database operations
- **Document Server**: Document management and search

## How to Use This Notebook

1. **Run cells ONE AT A TIME** (don't run all at once)
2. **Wait for each cell to complete** before running the next
3. **Look for execution numbers**: `[1]`, `[2]`, `[3]` means cell completed
4. **`[*]` means running**: Wait for it to finish

## Setup

Make sure you have installed the required packages:
```bash
pip install mcp jupyter
```

## Important: Understanding `await`

- Jupyter supports `await` directly in cells (no need for `asyncio.run()`)
- When you see `await`, the cell will:
  1. Start the MCP server automatically
  2. Connect to it
  3. Execute the command
  4. Show results
- If a cell hangs, press `Ctrl+C` (or `Cmd+C` on Mac) to interrupt

## Note: Execution Numbers

If you don't see cell execution numbers (like `[1]`, `[2]`) after running cells:
- **JupyterLab**: View → Show Line Numbers, Settings → enable `showExecutionTime`
- **Jupyter Notebook**: Execution numbers appear automatically
- See `NOTEBOOK_TROUBLESHOOTING.md` for detailed help


In [1]:
import asyncio
import json
import sys
from pathlib import Path
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# Get the Python interpreter from the current environment (venv)
# This ensures servers use the same Python that has MCP installed
PYTHON_EXECUTABLE = sys.executable
print(f"Using Python: {PYTHON_EXECUTABLE}")


Using Python: /Users/carbonjo/Library/CloudStorage/Dropbox/AI/Agents-Langchain-llamaindex-MCP/MCP_Nov29-25/venv/bin/python


## 1. Code Server Demo

The code server provides tools for file operations and code execution.


In [2]:
# Connect to the code server
# IMPORTANT: Use PYTHON_EXECUTABLE to ensure servers use the venv Python with MCP installed
code_server_params = StdioServerParameters(
    command=PYTHON_EXECUTABLE,  # Use venv Python, not system Python
    args=[str(Path("../servers/code_server.py").absolute())],
    env=None
)

async def demo_code_server():
    async with stdio_client(code_server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the session
            await session.initialize()
            
            print("=== Code Server Demo ===\n")
            
            # List available tools
            tools = await session.list_tools()
            print(f"Available tools: {[tool.name for tool in tools.tools]}\n")
            
            # Write a file
            print("1. Writing a test file...")
            result = await session.call_tool(
                "write_file",
                {"file_path": "test_script.py", "content": "print('Hello from MCP!')\nresult = 2 + 2\nprint(f'2 + 2 = {result}')"}
            )
            print(result.content[0].text if result.content else "No output")
            print()
            
            # Read the file
            print("2. Reading the file...")
            result = await session.call_tool("read_file", {"file_path": "test_script.py"})
            print(result.content[0].text if result.content else "No output")
            print()
            
            # Execute code
            print("3. Executing Python code...")
            result = await session.call_tool(
                "execute_code",
                {"code": "import math\nprint(f'Pi = {math.pi:.4f}')\nprint(f'Square root of 16 = {math.sqrt(16)}')"}
            )
            print(result.content[0].text if result.content else "No output")
            print()
            
            # Save a code snippet
            print("4. Saving a code snippet...")
            result = await session.call_tool(
                "save_code_snippet",
                {"name": "fibonacci", "code": "def fib(n):\n    if n <= 1:\n        return n\n    return fib(n-1) + fib(n-2)"}
            )
            print(result.content[0].text if result.content else "No output")
            print()
            
            # List code snippets
            print("5. Listing saved code snippets...")
            result = await session.call_tool("list_code_snippets", {})
            print(result.content[0].text if result.content else "No output")

# Run the demo
await demo_code_server()


=== Code Server Demo ===

Available tools: ['read_file', 'write_file', 'execute_code', 'save_code_snippet', 'list_code_snippets']

1. Writing a test file...
Successfully wrote 66 characters to test_script.py

2. Reading the file...
File contents of test_script.py:

print('Hello from MCP!')
result = 2 + 2
print(f'2 + 2 = {result}')

3. Executing Python code...
Execution result:
Pi = 3.1416
Square root of 16 = 4.0


4. Saving a code snippet...
Code snippet 'fibonacci' saved successfully

5. Listing saved code snippets...
Saved code snippets:
- fibonacci


## 2. Database Server Demo

The database server provides tools for SQLite database operations.


In [3]:
# Connect to the database server
db_server_params = StdioServerParameters(
    command=PYTHON_EXECUTABLE,  # Use venv Python, not system Python
    args=[str(Path("../servers/database_server.py").absolute())],
    env=None
)

async def demo_database_server():
    async with stdio_client(db_server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the session
            await session.initialize()
            
            print("=== Database Server Demo ===\n")
            
            # List available tools
            tools = await session.list_tools()
            print(f"Available tools: {[tool.name for tool in tools.tools]}\n")
            
            # List tables
            print("1. Listing database tables...")
            result = await session.call_tool("list_tables", {})
            print(result.content[0].text if result.content else "No output")
            print()
            
            # Describe a table
            print("2. Describing 'users' table...")
            result = await session.call_tool("describe_table", {"table_name": "users"})
            print(result.content[0].text if result.content else "No output")
            print()
            
            # Query all users
            print("3. Querying all users...")
            result = await session.call_tool("execute_query", {"query": "SELECT * FROM users"})
            print(result.content[0].text if result.content else "No output")
            print()
            
            # Insert a new user
            print("4. Inserting a new user...")
            result = await session.call_tool(
                "insert_user",
                {"name": "Diana Prince", "email": "diana@example.com", "age": 28}
            )
            print(result.content[0].text if result.content else "No output")
            print()
            
            # Get a specific user
            print("5. Getting user by email...")
            result = await session.call_tool("get_user", {"email": "diana@example.com"})
            print(result.content[0].text if result.content else "No output")
            print()
            
            # Query products
            print("6. Querying products...")
            result = await session.call_tool("execute_query", {"query": "SELECT * FROM products WHERE price < 100"})
            print(result.content[0].text if result.content else "No output")

# Run the demo
await demo_database_server()


=== Database Server Demo ===

Available tools: ['execute_query', 'list_tables', 'describe_table', 'insert_user', 'get_user']

1. Listing database tables...
Database tables:
- users
- sqlite_sequence
- products

2. Describing 'users' table...
Table 'users' schema:
- id (INTEGER)
- name (TEXT) NOT NULL
- email (TEXT) NOT NULL
- age (INTEGER)
- created_at (TIMESTAMP)

3. Querying all users...
Query results (3 rows):
[
  {
    "id": 1,
    "name": "Alice Johnson",
    "email": "alice@example.com",
    "age": 30,
    "created_at": "2025-11-30 00:46:02"
  },
  {
    "id": 2,
    "name": "Bob Smith",
    "email": "bob@example.com",
    "age": 25,
    "created_at": "2025-11-30 00:46:02"
  },
  {
    "id": 3,
    "name": "Charlie Brown",
    "email": "charlie@example.com",
    "age": 35,
    "created_at": "2025-11-30 00:46:02"
  }
]

4. Inserting a new user...
User 'Diana Prince' inserted successfully with ID 4

5. Getting user by email...
User found:
{
  "id": 4,
  "name": "Diana Prince",
  "e

## 3. Document Server Demo

The document server provides tools for document management and search.


In [4]:
# Connect to the document server
doc_server_params = StdioServerParameters(
    command=PYTHON_EXECUTABLE,  # Use venv Python, not system Python
    args=[str(Path("../servers/document_server.py").absolute())],
    env=None
)

async def demo_document_server():
    async with stdio_client(doc_server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # Initialize the session
            await session.initialize()
            
            print("=== Document Server Demo ===\n")
            
            # List available tools
            tools = await session.list_tools()
            print(f"Available tools: {[tool.name for tool in tools.tools]}\n")
            
            # Create a document
            print("1. Creating a document...")
            result = await session.call_tool(
                "create_document",
                {
                    "name": "introduction",
                    "content": "This is an introduction to MCP (Model Context Protocol).\nMCP allows AI agents to interact with various services through a standardized protocol.\nIt provides tools for code execution, database operations, and document management."
                }
            )
            print(result.content[0].text if result.content else "No output")
            print()
            
            # Create another document
            print("2. Creating another document...")
            result = await session.call_tool(
                "create_document",
                {
                    "name": "tutorial",
                    "content": "MCP Tutorial:\n\nStep 1: Set up an MCP server\nStep 2: Connect a client to the server\nStep 3: Use tools provided by the server\nStep 4: Access resources from the server"
                }
            )
            print(result.content[0].text if result.content else "No output")
            print()
            
            # List documents
            print("3. Listing all documents...")
            result = await session.call_tool("list_documents", {})
            print(result.content[0].text if result.content else "No output")
            print()
            
            # Read a document
            print("4. Reading a document...")
            result = await session.call_tool("read_document", {"name": "introduction"})
            print(result.content[0].text if result.content else "No output")
            print()
            
            # Search documents
            print("5. Searching for 'MCP' in documents...")
            result = await session.call_tool("search_documents", {"query": "MCP"})
            print(result.content[0].text if result.content else "No output")
            print()
            
            # Append to a document
            print("6. Appending to a document...")
            result = await session.call_tool(
                "append_to_document",
                {"name": "introduction", "content": "\n\nAdditional information: MCP servers can be written in any language that supports stdio communication."}
            )
            print(result.content[0].text if result.content else "No output")
            print()
            
            # Read the updated document
            print("7. Reading the updated document...")
            result = await session.call_tool("read_document", {"name": "introduction"})
            print(result.content[0].text if result.content else "No output")

# Run the demo
await demo_document_server()


=== Document Server Demo ===

Available tools: ['create_document', 'read_document', 'list_documents', 'search_documents', 'append_to_document', 'delete_document']

1. Creating a document...
Document 'introduction' created successfully (228 characters)

2. Creating another document...
Document 'tutorial' created successfully (164 characters)

3. Listing all documents...
Available documents (2):
- introduction
- tutorial

4. Reading a document...
Document 'introduction':

This is an introduction to MCP (Model Context Protocol).
MCP allows AI agents to interact with various services through a standardized protocol.
It provides tools for code execution, database operations, and document management.

5. Searching for 'MCP' in documents...
Search results for 'MCP':
- introduction (2 occurrence(s))
- tutorial (2 occurrence(s))

6. Appending to a document...
Appended 103 characters to 'introduction'

7. Reading the updated document...
Document 'introduction':

This is an introduction to MCP (M

## 4. Combined Workflow Demo

This demonstrates using all three servers together in a realistic workflow.


In [5]:
async def combined_workflow():
    print("=== Combined Workflow Demo ===\n")
    print("Scenario: Analyzing user data and generating a report\n")
    
    # Step 1: Query database for user statistics
    print("Step 1: Querying database for user statistics...")
    async with stdio_client(db_server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            result = await session.call_tool(
                "execute_query",
                {"query": "SELECT COUNT(*) as total_users, AVG(age) as avg_age FROM users"}
            )
            stats = result.content[0].text if result.content else ""
            print(stats)
            print()
    
    # Step 2: Generate a Python script to process the data
    print("Step 2: Generating a data processing script...")
    async with stdio_client(code_server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            script_content = """
# User Statistics Report
import json

data = {
    'total_users': 4,
    'avg_age': 29.5,
    'report_date': '2024-01-01'
}

print('User Statistics Report')
print('=' * 30)
print(json.dumps(data, indent=2))
"""
            
            result = await session.call_tool(
                "write_file",
                {"file_path": "user_report.py", "content": script_content}
            )
            print(result.content[0].text if result.content else "")
            print()
            
            # Execute the script
            result = await session.call_tool(
                "execute_code",
                {"code": script_content}
            )
            print("Execution result:")
            print(result.content[0].text if result.content else "")
            print()
    
    # Step 3: Save the report as a document
    print("Step 3: Saving report as a document...")
    async with stdio_client(doc_server_params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            report_content = """User Statistics Report
Generated: 2024-01-01

Summary:
- Total Users: 4
- Average Age: 29.5 years

This report was generated using MCP agents to query the database,
process the data, and create documentation.
"""
            
            result = await session.call_tool(
                "create_document",
                {"name": "user_report", "content": report_content}
            )
            print(result.content[0].text if result.content else "")
            print()
            
            # Search for the report
            result = await session.call_tool("search_documents", {"query": "report"})
            print("Search results for 'report':")
            print(result.content[0].text if result.content else "")

# Run the combined workflow
await combined_workflow()


=== Combined Workflow Demo ===

Scenario: Analyzing user data and generating a report

Step 1: Querying database for user statistics...
Query results (1 rows):
[
  {
    "total_users": 4,
    "avg_age": 29.5
  }
]

Step 2: Generating a data processing script...
Successfully wrote 208 characters to user_report.py

Execution result:
Execution result:
User Statistics Report
{
  "total_users": 4,
  "avg_age": 29.5,
  "report_date": "2024-01-01"
}


Step 3: Saving report as a document...
Document 'user_report' created successfully (209 characters)

Search results for 'report':
Search results for 'report':
- user_report (2 occurrence(s))


## Summary

This demo has shown:

1. **Code Server**: File operations, code execution, and code snippet management
2. **Database Server**: SQL queries, table management, and data operations
3. **Document Server**: Document creation, reading, searching, and management
4. **Combined Workflow**: Using all three servers together for a complete task

MCP provides a standardized way for AI agents to interact with various services, making it easy to build powerful agentic applications!
